In [39]:
import smtplib
import datetime
import pytz
from email.mime.text import MIMEText
import time
import os
import json
import threading
from IPython.display import Audio, display


In [40]:
REMINDERS_FILE = "reminders.json"
SENDER_EMAIL = " " #your email
EMERGENCY_EMAIL_Healthcare = " " #healthcentre's Email
EMERGENCY_EMAIL_Fire = " "#fire's Email
EMERGENCY_EMAIL_Guard = " "#guard's Email
PASSWORD = " " # APP password '16 digit pin'
os.system('wget -O reminder_sound.mp3 https://cdn.pixabay.com/download/audio/2022/03/15/audio_8cb749d484.mp3')

0

In [41]:

def get_indian_time():
    """Get current date and time in Indian timezone (IST)"""
    return datetime.datetime.now(pytz.timezone('Asia/Kolkata'))



In [42]:
def send_reminder_email(recipient_email, event_datetime, event_name):
    """Sends a reminder email to the specified recipient."""
    event_date_str = event_datetime.strftime("%A, %B %d, %Y at %I:%M %p")
    subject = f"Reminder: {event_name} tomorrow"
    message = f"""
Hello,

This is a friendly reminder that you have "{event_name}" scheduled for tomorrow, {event_date_str}.

Don't forget!

Best regards,
Your Reminder Service
"""
    msg = MIMEText(message)
    msg['Subject'], msg['From'], msg['To'] = subject, SENDER_EMAIL, recipient_email

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(SENDER_EMAIL, PASSWORD)
            server.sendmail(SENDER_EMAIL, recipient_email, msg.as_string())
        print(f"Reminder email sent to {recipient_email}")
        return True
    except Exception as e:
        print(f"Failed to send email: {e}")
        return False

In [43]:
def send_audio_notification(event_name, event_datetime):
    """Plays a longer audio notification for a reminder."""
    event_date_str = event_datetime.strftime("%A, %B %d, %Y at %I:%M %p")

    # Play the downloaded alarm sound (15 seconds)
    display(Audio("reminder_sound.mp3", autoplay=True))

    print(f"⏰ REMINDER ALERT ⏰")
    print(f"Event: {event_name}")
    print(f"Date: {event_date_str}")
    print(f"Don't forget this important event!")
    return True

In [44]:
def save_reminders(reminders):
    """Save reminders to file"""
    serializable_reminders = [{**r, 'event_datetime': r['event_datetime'].isoformat(),
                               'reminder_datetime': r['reminder_datetime'].isoformat()} for r in reminders]
    with open(REMINDERS_FILE, 'w') as f:
        json.dump(serializable_reminders, f)

In [45]:
def load_reminders():
    """Load saved reminders from file"""
    if not os.path.exists(REMINDERS_FILE):
        return []
    try:
        with open(REMINDERS_FILE, 'r') as f:
            reminders = json.load(f)
        for r in reminders:
            r['event_datetime'] = datetime.datetime.fromisoformat(r['event_datetime'])
            r['reminder_datetime'] = datetime.datetime.fromisoformat(r['reminder_datetime'])
        return reminders
    except:
        return []

In [46]:
def check_date(event_datetime, reminder_datetime):
    current_time = get_indian_time()

    if event_datetime <= current_time:
        return False, "Event date has already passed."
    elif reminder_datetime <= current_time:
        return False, "Reminder time has already passed."
    else:
        return True, "Date and time are valid."

In [47]:
def add_reminder(user_email, event_name, event_datetime):
    """Add a new reminder to the system."""
    time_before = input("Enter hours and minutes before event (format: hours:minutes): ")
    hours_before, minutes_before = map(int, time_before.split(':'))

    total_minutes = (hours_before * 60) + minutes_before
    reminder_datetime = event_datetime - datetime.timedelta(minutes=total_minutes)

    is_valid, message = check_date(event_datetime, reminder_datetime)
    if not is_valid:
        print(message)
        return

    reminder = {
        'user_email': user_email,
        'event_name': event_name,
        'event_datetime': event_datetime,
        'reminder_datetime': reminder_datetime,
        'sent': False
    }
    audio_reminder = {
        'user_email': user_email,
        'event_name': event_name,
        'event_datetime': event_datetime,
        'reminder_datetime': event_datetime - datetime.timedelta(minutes=2),
        'sent': False,
        'method': 'audio'
    }

    reminders = load_reminders()
    reminders.append(reminder)
    reminders.append(audio_reminder)
    save_reminders(reminders)

    print(f"Reminder added for {event_name} on {event_datetime.strftime('%Y-%m-%d %H:%M')} IST")
    print(f"Email reminder will be sent on {reminder_datetime.strftime('%Y-%m-%d %H:%M')} IST")
    print(f"Audio alert will ring 2 minutes before the event")

In [48]:
def check_reminders():
    """Check for reminders that need to be sent"""
    current_time = get_indian_time()
    reminders = load_reminders()
    updated = False

    for reminder in reminders:
        if not reminder['sent'] and current_time >= reminder['reminder_datetime']:
            print(f"Sending reminder for {reminder['event_name']}...")

            if reminder.get('method') == 'audio':
                if send_audio_notification(reminder['event_name'], reminder['event_datetime']):
                    reminder['sent'] = True
                    updated = True
            else:
                if send_reminder_email(reminder['user_email'], reminder['event_datetime'], reminder['event_name']):
                    reminder['sent'] = True
                    updated = True

    if updated:
        save_reminders(reminders)

In [49]:
def Fetch_live_location:
  return Live_location

SyntaxError: expected '(' (<ipython-input-49-56bd62372212>, line 1)

In [50]:
Live_location = "   fill me🥲.  " #fill this part
def Emergency_special(EMERGENCY_EMAIL, event_datetime,):
    event_date_str = event_datetime.strftime("%A, %B %d, %Y at %I:%M %p")
    subject = "Emergency Emergency!"
    message = f"""
Hello,

This is an Emergency in the campus at {Live_location}
Your Reminder Service
"""
    msg = MIMEText(message)
    msg['Subject'], msg['From'], msg['To'] = subject, SENDER_EMAIL, EMERGENCY_EMAIL

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(SENDER_EMAIL, PASSWORD)
            server.sendmail(SENDER_EMAIL, EMERGENCY_EMAIL, msg.as_string())
        print(f"Emergency mail email sent to {EMERGENCY_EMAIL}")
        return True
    except Exception as e:
        print(f"Failed to send email: {e}")

In [63]:
def reminder_checker():
    """Function to continuously check reminders"""
    while True:
        check_reminders()
        time.sleep(60) # Check every minute

# Start automatic reminder checker in a background thread
checker_thread = threading.Thread(target=reminder_checker, daemon=True)
checker_thread.start()
print(" Incase of Emergency don't fill email just press Enter for faster services")
# Interactive part
user_email = input("Enter your email address: ")
print("Automatic reminder checking is now running in the background.")
print("Reminders will be checked every minute.")

while True:
    print("\nReminder System Menu:")
    print("1. Add a reminder")
    print("2. Emergency")
    print("3. Exit")
    choice = input("Enter your choice (1-2-3): ")

    if choice == '1':
        event_name = input("Enter event name: ")
        event_date_str = input("Enter event date (YYYY-MM-DD): ")
        event_time_str = input("Enter event time (HH:MM): ")

        try:
            naive_datetime = datetime.datetime.strptime(f"{event_date_str} {event_time_str}", "%Y-%m-%d %H:%M")
            event_datetime = pytz.timezone('Asia/Kolkata').localize(naive_datetime)
            add_reminder(user_email, event_name, event_datetime)
        except ValueError:
            print("Invalid date or time format. Please try again.")

    elif choice == '2':
        choose = input("What is your emergency related to? \n if health type :1 \n if Fire type :2 \n if Guard type :3.")
        if choose == '1':
            EMERGENCY_EMAIL = EMERGENCY_EMAIL_Healthcare
        elif choose == '2':
            EMERGENCY_EMAIL = EMERGENCY_EMAIL_Fire
        elif choose == '3':
            EMERGENCY_EMAIL = EMERGENCY_EMAIL_Guard
        else:
            print("Invalid choice. Please try again.")

        event_name = "Emergency"
        event_datetime = datetime.datetime.now(pytz.timezone('Asia/Kolkata'))
        event_date_str = event_datetime.strftime("%Y-%m-%d %H:%M")
        naive_datetime = datetime.datetime.strptime(f"{event_date_str}", "%Y-%m-%d %H:%M")
        event_datetime = pytz.timezone('Asia/Kolkata').localize(naive_datetime)
        Emergency_special(EMERGENCY_EMAIL, event_datetime)

    elif choice == '3':
        print("Exiting reminder system.")
        break

    else:
        print("Invalid choice. Please try again.")


Enter your email address: 
Automatic reminder checking is now running in the background.
Reminders will be checked every minute.

Reminder System Menu:
1. Add a reminder
2. Emergency
3. Exit
Enter your choice (1-2-3): 2


KeyboardInterrupt: Interrupted by user

datetime.datetime(2025, 3, 21, 14, 39, 32, 71215, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>)